In [1]:
%run basics
%matplotlib

Using matplotlib backend: Qt4Agg


In [2]:
def NEE_Lasslop(Fsd,D,T,D0,E0,alpha,beta0,k,rb):
    NEE = -1*GPP_Lasslop(Fsd,D,alpha,beta0,k,D0) + ER_LloydTaylor(T,rb,E0)
    return NEE

In [3]:
def NEE_Isaac(Fsd,D,T,D0,E0,alpha,beta0,rb):
#def NEE_Isaac(args,params):
    NEE = -1*GPP_Isaac(Fsd,D,alpha,beta0,D0) + ER_LloydTaylor(T,rb,E0)
    return NEE

In [4]:
def NEE_NRH_Isaac(Fsd,D,T,D0,E0,alpha,beta0,eta,rb):
    NEE = -1*GPP_NRH_Isaac(Fsd,D,alpha,beta0,eta,D0) + ER_LloydTaylor(T,rb,E0)
    return NEE

In [5]:
def GPP_Lasslop(Fsd,D,alpha,beta0,k,D0):
    beta = beta0*SHD_func_Lasslop(D,k,D0)
    GPP = alpha*beta*Fsd/(alpha*Fsd+beta)
    return GPP

In [6]:
def GPP_Isaac(Fsd,D,alpha,beta0,D0):
    beta = beta0*SHD_func_Isaac(D,D0)
    GPP = alpha*beta*Fsd/(alpha*Fsd+beta)
    return GPP

In [7]:
def GPP_NRH_Isaac(Fsd,D,alpha,beta0,eta,D0):
    beta = beta0*SHD_func_Isaac(D,D0)
    GPP = (1/(2*eta))*(alpha*Fsd+beta-numpy.sqrt((alpha*Fsd+beta)**2-4*alpha*beta*eta*Fsd))
    return GPP

In [8]:
def SHD_func_Lasslop(D,k,D0):
    SHD_func = numpy.ones(len(D))
    idx = numpy.where(D>D0)[0]
    SHD_func[idx] = numpy.exp(-k*(D[idx]-D0))
    return SHD_func

In [9]:
def SHD_func_Isaac(D,D0):
    SHD_func = numpy.ones(len(D))
    idx = numpy.where(D>D0)[0]
    SHD_func[idx] = 1/(1+(D[idx]-D0)/D0)
    return SHD_func

In [10]:
def ER_LloydTaylor(T,rb,E0):
#    return rb*numpy.exp(E0*(1/(c.Tref-c.T0)-1/(T-c.T0)))
    return rb*numpy.exp(E0*(1/(288.13-227.13)-1/((T+273.15)-227.13)))

In [11]:
def plot_ER(fig,i,ER,T,Sws,weight_type='Huber'):
    ax = fig.add_subplot(2,2,i)
    popt,pcov = irls_leastsq(residuals_LT,[2,200],args=(ER,T),maxfev=3,weight_type=weight_type)
    resid = ER - ER_LloydTaylor(T,popt[0],popt[1])
    weights = CalculateWeights(resid,weight_type=weight_type)
    plt.scatter(T,ER,c=weights)
    #mean,edges,number = scipy.stats.binned_statistic(T,Reco,statistic='mean')
    #mid = [(edges[i]+edges[i+1])/2 for i in range(0,len(edges)-1)]
    #plt.plot(mid,mean,'ro')
    Xfit = numpy.linspace(numpy.min(T),numpy.max(T),100)
    Yfit = ER_LloydTaylor(Xfit,popt[0],popt[1])
    plt.plot(Xfit,Yfit,'r-')
    text_str = ('%.2f < Sws < %.2f'%(Sws[0],Sws[-1]))
    plt.text(0.5,0.9,text_str,horizontalalignment='center',transform=ax.transAxes)
    text_str = ('rb=%.2f,E0=%.2f'%(popt[0],popt[-1]))
    plt.text(0.5,0.8,text_str,horizontalalignment='center',transform=ax.transAxes)

In [12]:
def irls_leastsq(func,p0,args=(),maxfev=3,weight_type='Huber',mode='verbose'):
    weighted = False
    popt,pcov = scipy.optimize.leastsq(func,p0,args=args+(weighted,weight_type))
    print 'After non-weighted call to leastsq: ',popt
    n=1
    weighted = True
    while n<=maxfev:
        popt,pcov = scipy.optimize.leastsq(func,popt,args=args+(weighted,weight_type))
        if mode=='verbose':
            print 'Weighted call '+str(n)+' to leastsq: ',popt
        ER_fit = ER_LloydTaylor(args[1],popt[0],popt[1])
        #NEE_plt = NEE_Isaac(args[1:],popt)
        resid = args[0] - ER_fit
        weights = CalculateWeights(resid,weight_type=weight_type)
        #fig = plt.figure()
        #plt.scatter(args[1],args[0],c=weights)
        #plt.plot(args[1],ER_fit,'r+')
        #plt.show()
        n = n + 1
    return popt,pcov

In [13]:
def residuals_LloydTaylor(params,ER,T,weighted,weight_type):
    rb = params[0]
    E0 = params[1]
    resid = ER - ER_LloydTaylor(T,rb,E0)
    if weighted:
        weights = CalculateWeights(resid,weight_type=weight_type)
        return resid*weights
    else:
        return resid

In [14]:
def residuals_Lasslop(params,NEE,Fsd,D,T,D0,E0,weighted,weight_type):
    alpha = params[0]
    beta0 = params[1]
    k = params[2]
    rb = params[3]
    resid = NEE - NEE_Lasslop(Fsd,D,T,D0,E0,alpha,beta0,k,rb)
    if weighted:
        weights = CalculateWeights(resid,weight_type=weight_type)
        return resid*weights
    else:
        return resid

In [15]:
def residuals_Isaac(params,NEE,Fsd,D,T,D0,E0,weighted,weight_type):
    """
    Purpose:
    Residual function for use in fitting LRC to observations of NEE.
    Usage:
    Passed to leastsq curve fit function.
    Where:
     params is an array of paramters to be fitted
      - params[0] is alpha
        params[1] is beta0
        params[2] is eta, curvature of NRH
        params[3] is rb
    NEE is observed net ecosystem exchange
    Fsd is incoming shortwave
    D is SHD
    T is T
    D0 is blah
    E0 is temperature sensitivity of ER
    Author: PRI
    Date: Back in the day
    """
    alpha = params[0]
    beta0 = params[1]
    eta = params[2]
    rb = params[3]
    resid = NEE - NEE_NRH_Isaac(Fsd,D,T,D0,E0,alpha,beta0,eta,rb)
    if weighted:
        weights = CalculateWeights(resid,weight_type=weight_type)
        return resid*weights
    else:
        return resid

In [16]:
def CalculateWeights(resid,weight_type='Huber'):
    if weight_type.lower()=='tukey':
        weights = TukeyBiweight(MAD_scale(resid))
    elif weight_type.lower()=='huber':
        weights = HuberTWeight(MAD_scale(resid))
    elif weight_type.lower()=='ols':
        weights = OLSWeight(MAD_scale(resid))
    else:
        print "CalculateWeights: Unknown weight type, used Huber's t weight"
        weights = HuberTWeight(MAD_scale(resid))
    return weights

In [17]:
def TukeyBiweight(x):
    return ((numpy.abs(x)<1.0)*(1.0 - x**2)**2)

In [18]:
def HuberTWeight(x):
    weights = numpy.ones(len(x))
    idx = numpy.where(numpy.abs(x)>1.345)[0]
    weights[idx] = 1.345/numpy.abs(x[idx])
    return weights

In [19]:
def OLSWeight(x):
    weights = numpy.ones(len(x))
    return weights

In [20]:
def MAD_scale(resid):
    scale = numpy.median(numpy.abs(resid - numpy.median(resid))) / 0.6745
    return resid/scale

In [21]:
#ncname = "qcio.get_filename_dialog(path="../../Sites/HowardSprings/Data/Processed/all")"
ncname ="C:/OzFlux/Sites/CUP/Processed/CUP_2012_to_2015_L6_v295.nc"
print ncname
ds = qcio.nc_read_series(ncname)
ldt = ds.series["DateTime"]["Data"]
ts = int(ds.globalattributes["time_step"])
site_name = ds.globalattributes["site_name"]

C:/OzFlux/Sites/CUP/Processed/CUP_2012_to_2015_L6_v295.nc
C:/OzFlux/Sites/CUP/Processed/CUP_2012_to_2015_L6_v295.nc


In [78]:
Fsd,f,a = qcutils.GetSeriesasMA(ds,"Fsd")
D,f,a = qcutils.GetSeriesasMA(ds,"VPD")
T,f,a = qcutils.GetSeriesasMA(ds,"Ta")
ER,f,a = qcutils.GetSeriesasMA(ds,"ER")
Fc,f,a = qcutils.GetSeriesasMA(ds,"Fc")

In [79]:
fig = plt.figure(1)
ax1=plt.subplot(511)
plt.plot(ldt,Fsd)
ax2=plt.subplot(512,sharex=ax1)
plt.plot(ldt,D)
ax3=plt.subplot(513,sharex=ax1)
plt.plot(ldt,T)
ax4=plt.subplot(514,sharex=ax1)
plt.plot(ldt,Fc)
ax5=plt.subplot(515,sharex=ax1)
plt.plot(ldt,ER)
plt.show()

In [80]:
mask = ER.mask
for item in [T]:
    mask = numpy.ma.mask_or(mask,item.mask)
ER = numpy.ma.array(ER,mask=mask)
T = numpy.ma.array(T,mask=mask)
start_date = ldt[0]
end_date = ldt[-1]
this_date = start_date+dateutil.relativedelta.relativedelta(months=1)
LloydTaylor_results = {"start_date":[],"end_date":[],"rb":[],"E0":[]}
while start_date<end_date:
    LloydTaylor_results["start_date"].append(str(start_date))
    LloydTaylor_results["end_date"].append(str(end_date))
    si = qcutils.GetDateIndex(ldt,str(start_date))
    ei = qcutils.GetDateIndex(ldt,str(end_date))
    ER_plt = numpy.ma.compressed(ER[si:ei+1])
    T_plt = numpy.ma.compressed(T[si:ei+1])
    popt,pcov = irls_leastsq(residuals_LloydTaylor,[1,100],args=(ER_plt,T_plt),maxfev=3,weight_type='Huber',mode='quiet')
    LloydTaylor_results["rb"].append(popt[0])
    LloydTaylor_results["E0"].append(popt[1])
    start_date = this_date
    this_date = start_date+dateutil.relativedelta.relativedelta(months=1)

After non-weighted call to leastsq:  [   1.90137294  234.94969453]
After non-weighted call to leastsq:  [   1.90151661  236.05555074]
After non-weighted call to leastsq: After non-weighted call to leastsq:  [   1.90137294  234.94969453]
After non-weighted call to leastsq:  [   1.90151661  236.05555074]
After non-weighted call to leastsq:  [   1.9038782   236.22424634]
After non-weighted call to leastsq:  [   1.9038782   236.22424634]
After non-weighted call to leastsq:  [   1.89636458  251.74429263]
After non-weighted call to leastsq:  [   1.89636458  251.74429263]
After non-weighted call to leastsq:  [   1.887009    255.62671028]
After non-weighted call to leastsq:  [   1.88727837  249.21668691]
After non-weighted call to leastsq:  [   1.887009    255.62671028]
After non-weighted call to leastsq:  [   1.88727837  249.21668691]
After non-weighted call to leastsq:  [   1.88494721  249.09759354]
After non-weighted call to leastsq:  [   1.88494721  249.09759354]
After non-weighted call to

In [83]:
for i in range(len(LloydTaylor_results["start_date"])):
    print LloydTaylor_results["start_date"][i],LloydTaylor_results["rb"][i],LloydTaylor_results["E0"][i]
    

2012-10-19 00:00:00 1.71838719928 190.991662073
2012-11-19 00:00:00 1.75809289364 183.771030119
2012-12-19 00:00:00 1.72937790381 183.288415394
2013-01-19 00:00:00 1.61914923786 245.880950688
2013-02-19 00:00:00 1.63588619405 220.231507857
2013-03-19 00:00:00 1.77112622133 188.440251597
2013-04-19 00:00:00 1.88495554551 249.116688738
2013-05-19 00:00:00 1.59115247154 213.29218354
2013-06-19 00:00:00 1.68123162967 200.754857405
2013-07-19 00:00:00 1.59041268755 213.723727543
2013-08-19 00:00:00 1.73395935987 238.61112621
2013-09-19 00:00:00 1.75071088632 230.235885282
2013-10-19 00:00:00 1.46831363884 374.183817642
2013-11-19 00:00:00 1.91103874715 175.898874551
2013-12-19 00:00:00 1.68278870946 202.259860686
2014-01-19 00:00:00 1.58211683395 325.190899068
2014-02-19 00:00:00 1.56571716234 333.97287778
2014-03-19 00:00:00 1.55593129194 327.873427044
2014-04-19 00:00:00 1.61688033184 274.72063704
2014-05-19 00:00:00 1.57668420028 261.655739364
2014-06-19 00:00:00 1.698083809 290.71033090

In [81]:
fig=plt.figure()
plt.plot(LloydTaylor_results["E0"])
plt.show()

In [73]:
Fsd,f,a = qcutils.GetSeriesasMA(ds,"Fsd")
D,f,a = qcutils.GetSeriesasMA(ds,"VPD")
T,f,a = qcutils.GetSeriesasMA(ds,"Ta")
ER,f,a = qcutils.GetSeriesasMA(ds,"ER")
Fc,f,a = qcutils.GetSeriesasMA(ds,"Fc")

In [74]:
Fc = numpy.ma.masked_where(f!=0,Fc)
#see the beautiful way to mask the data!
Fc_day = numpy.ma.masked_where(Fsd<=10,Fc)
Fc_night = numpy.ma.masked_where(Fsd>10,Fc)
mask = Fc_day.mask
for item in [D,T]:
    mask = numpy.ma.mask_or(mask,item.mask)
NEE_day = numpy.ma.array(Fc_day,mask=mask)
Fsd = numpy.ma.array(Fsd,mask=mask)
D = numpy.ma.array(D,mask=mask)
T = numpy.ma.array(T,mask=mask)

In [75]:
fig = plt.figure(1)
ax1=plt.subplot(411)
plt.plot(ldt,Fsd)
ax2=plt.subplot(412,sharex=ax1)
plt.plot(ldt,D)
ax3=plt.subplot(413,sharex=ax1)
plt.plot(ldt,T)
ax4=plt.subplot(414,sharex=ax1)
plt.plot(ldt,NEE_day)
plt.show()

In [76]:
window_size = 15
jump_size = 5
start_date = ldt[0]
#end_date = datetime.datetime(start_date.year+1,1,1)+datetime.timedelta(minutes=ts)
end_date = ldt[-1]
this_date = start_date + datetime.timedelta(days=window_size)
Lasslop_results = {"start_date":[],"end_date":[],"alpha":[],"beta0":[],"k":[],"rb_day":[]}
while this_date<end_date:
    Lasslop_results["start_date"].append(str(start_date))
    Lasslop_results["end_date"].append(str(end_date))
    si = qcutils.GetDateIndex(ldt,str(start_date),ts=ts)
    ei = qcutils.GetDateIndex(ldt,str(this_date),ts=ts)
    ldt_plt = ldt[si:ei+1]
    NEE_plt = NEE_day[si:ei+1]
    Fsd_plt = Fsd[si:ei+1]
    D_plt = D[si:ei+1]
    T_plt = T[si:ei+1]
    #this identifies index values or row numbers of good data
    idx = numpy.ma.where(NEE_plt.mask==False)[0]
    ldt_plt = [ldt_plt[i] for i in idx]
    #numpy.ma.compressed tosses out the masked data in order to curve fit on the subset
    NEE_plt = numpy.ma.compressed(NEE_plt)
    Fsd_plt = numpy.ma.compressed(Fsd_plt)
    D_plt = numpy.ma.compressed(D_plt)
    T_plt = numpy.ma.compressed(T_plt)
    #popt,pcov = irls_leastsq(residuals_Lasslop,[0.1,100,0,1],args=(NEE_plt,Fsd_plt,D_plt,T_plt,0.005,210),maxfev=3,weight_type='Huber')
    popt,pcov = irls_leastsq(residuals_Isaac,[0.1,100,1,1],args=(NEE_plt,Fsd_plt,D_plt,T_plt,1.0,250),
                             maxfev=3,weight_type='Huber',mode='quiet')
    Lasslop_results["alpha"].append(popt[0])
    Lasslop_results["beta0"].append(popt[1])
    Lasslop_results["k"].append(popt[2])
    Lasslop_results["rb_day"].append(popt[3])
    #print start_date,this_date,popt
    start_date = start_date + datetime.timedelta(days=jump_size)
    this_date = start_date + datetime.timedelta(days=window_size)

After non-weighted call to leastsq:  [  1.02094644e-02   1.12188420e+01   1.00000556e+00   2.40383036e-01]
After non-weighted call to leastsq: After non-weighted call to leastsq:  [  1.02094644e-02   1.12188420e+01   1.00000556e+00   2.40383036e-01]
After non-weighted call to leastsq:  [  8.98820644e-03   1.04516309e+01   1.00000853e+00   3.91315559e-01]
After non-weighted call to leastsq:  [  8.98820644e-03   1.04516309e+01   1.00000853e+00   3.91315559e-01]
After non-weighted call to leastsq:  [  7.47390884e-03   1.09331010e+01   1.00000013e+00   5.10170927e-01]
After non-weighted call to leastsq:  [  7.47390884e-03   1.09331010e+01   1.00000013e+00   5.10170927e-01]
After non-weighted call to leastsq:  [  8.66272671e-03   1.26748513e+01   9.77304802e-01   7.36171931e-01]
After non-weighted call to leastsq:  [  8.66272671e-03   1.26748513e+01   9.77304802e-01   7.36171931e-01]
After non-weighted call to leastsq:  [  8.41836817e-03   1.29112276e+01   9.79458209e-01   4.93803002e-01]
A

C:\Anaconda\lib\site-packages\IPython\kernel\__main__.py:3: RuntimeWarning: invalid value encountered in sqrt
  app.launch_new_instance()
C:\Anaconda\lib\site-packages\IPython\kernel\__main__.py:2: RuntimeWarning: invalid value encountered in absolute
  from IPython.kernel.zmq import kernelapp as app
C:\Anaconda\lib\site-packages\IPython\kernel\__main__.py:3: RuntimeWarning: invalid value encountered in absolute
  app.launch_new_instance()
C:\Anaconda\lib\site-packages\IPython\kernel\__main__.py:3: RuntimeWarning: invalid value encountered in greater
  app.launch_new_instance()
C:\Anaconda\lib\site-packages\IPython\kernel\__main__.py:3: RuntimeWarning: overflow encountered in exp
  app.launch_new_instance()
C:\Anaconda\lib\site-packages\IPython\kernel\__main__.py:3: RuntimeWarning: invalid value encountered in sqrt
  app.launch_new_instance()
C:\Anaconda\lib\site-packages\IPython\kernel\__main__.py:2: RuntimeWarning: invalid value encountered in absolute
  from IPython.kernel.zmq impor

 [  1.23623549e-02   4.81914454e+05   1.91040576e+03   4.47942302e-02]
After non-weighted call to leastsq:  [  1.23623549e-02   4.81914454e+05   1.91040576e+03   4.47942302e-02]
After non-weighted call to leastsq:  [ nan  nan  nan  nan]
After non-weighted call to leastsq:  [ nan  nan  nan  nan]
After non-weighted call to leastsq:  [  4.66826815e-06  -2.61283662e-03   4.36254122e-04  -5.33499127e+00]
After non-weighted call to leastsq:  [  4.66826815e-06  -2.61283662e-03   4.36254122e-04  -5.33499127e+00]
After non-weighted call to leastsq:  [  2.82918713e-11  -1.85567159e-08   3.52807883e-09  -5.51854023e+00]
After non-weighted call to leastsq:  [  6.83553135e-04  -1.87762710e-01   3.42491656e-02  -4.54873939e+00]
After non-weighted call to leastsq:  [  2.82918713e-11  -1.85567159e-08   3.52807883e-09  -5.51854023e+00]
After non-weighted call to leastsq:  [  6.83553135e-04  -1.87762710e-01   3.42491656e-02  -4.54873939e+00]
After non-weighted call to leastsq:  [  6.32067806e-03   7.454

C:\Anaconda\lib\site-packages\IPython\kernel\__main__.py:2: RuntimeWarning: invalid value encountered in subtract
  from IPython.kernel.zmq import kernelapp as app


 [  0.0123019   12.16841808   0.99872128   0.72099106]
After non-weighted call to leastsq:  [  0.01717251  15.91519377   0.74211431   1.04550225]


C:\Anaconda\lib\site-packages\IPython\kernel\__main__.py:2: RuntimeWarning: invalid value encountered in subtract
  from IPython.kernel.zmq import kernelapp as app


In [77]:
fig=plt.figure()
plt.plot(Lasslop_results["rb_day"])
plt.show()

In [42]:
fig=plt.figure(1)
ax1=plt.subplot(411)
plt.plot(ldt_plt,Fsd_plt,'b.')
ax2=plt.subplot(412,sharex=ax1)
plt.plot(ldt_plt,D_plt,'b.')
ax3=plt.subplot(413,sharex=ax1)
plt.plot(ldt_plt,T_plt,'b.')
ax4=plt.subplot(414,sharex=ax1)
plt.plot(ldt_plt,NEE_plt,'b.')
plt.show()

In [43]:
fig=plt.figure(1)
plt.plot(Fsd_plt,NEE_plt,'b.')
plt.show()

In [44]:
fig=plt.figure()
plt.plot(D_plt,SHD_func_Isaac(D_plt,0.005),'b.')
plt.show()